#Bibliotecas

In [ ]:
import os
import cv2
import numpy as np
import torch
import pickle
from torchvision import transforms
from tqdm import tqdm

#Montando o Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Diretórios e Definições

In [ ]:
# Diretórios do KITTI Dataset
BASE_PATH = "/content/drive/My Drive/Sistemas Mecatrônicos/Cibernética e Aprendizagem de Máquinas/Projeto2/data_odometry_color/dataset/sequences"
BASE_LIDAR = "/content/drive/My Drive/Sistemas Mecatrônicos/Cibernética e Aprendizagem de Máquinas/Projeto2/data_odometry_velodyne/dataset/sequences"
BASE_POSES = "/content/drive/My Drive/Sistemas Mecatrônicos/Cibernética e Aprendizagem de Máquinas/Projeto2/data_odometry_poses/dataset/poses"
SAVE_PATH = "/content/drive/My Drive/Sistemas Mecatrônicos/Cibernética e Aprendizagem de Máquinas/Projeto2/processed_data"  # Pasta onde os dados serão salvos

# Criar a pasta de saída se não existir
os.makedirs(SAVE_PATH, exist_ok=True)

# Definir transformações para normalizar imagens
image_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),  # Redimensiona diretamente o tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#Função de Load de Imagens

In [ ]:
# Função para carregar e normalizar imagens RGB
def load_images(sequence):
    img_path = os.path.join(BASE_PATH, sequence, "image_2").replace("\\", "/")
    img_files = sorted(os.listdir(img_path))
    images = []

    for img_file in tqdm(img_files, desc=f"Carregando imagens da sequência {sequence}"):
        img = cv2.imread(os.path.join(img_path, img_file).replace("\\", "/"))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = image_transform(img)  # Normaliza a imagem
        images.append(img)

    return torch.stack(images)  # Retorna tensor (N, C, H, W)

#Função de Load de dados do LiDAR

In [ ]:
# Função para carregar e normalizar dados LiDAR
def load_lidar(sequence):
    lidar_path = os.path.join(BASE_LIDAR, sequence, "velodyne").replace("\\", "/")
    lidar_files = sorted(os.listdir(lidar_path))
    lidar_data = []

    for lidar_file in tqdm(lidar_files, desc=f"Carregando LiDAR da sequência {sequence}"):
        bin_path = os.path.join(lidar_path, lidar_file).replace("\\", "/")
        points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)  # (X, Y, Z, Intensidade)

        # Normaliza XYZ para intervalo [-1, 1] com base no valor máximo absoluto
        max_abs = np.max(np.abs(points[:, :3]))
        points[:, :3] = points[:, :3] / max_abs if max_abs > 0 else points[:, :3]

        lidar_data.append(points[:, :3])  # Mantém apenas XYZ

    return lidar_data  # Lista de numpy arrays

#Função de Load de dados de posição (ground truth)

In [ ]:
# Função para carregar poses (matrizes de transformação 4x4)
def load_poses(sequence):
    pose_file = os.path.join(BASE_POSES, f"{sequence}.txt").replace("\\", "/")
    poses = []

    with open(pose_file, "r") as f:
        for line in f.readlines():
            pose_values = np.array([float(x) for x in line.strip().split()]).reshape(3, 4)
            pose_matrix = np.vstack((pose_values, [0, 0, 0, 1]))  # Adiciona linha para matriz 4x4
            poses.append(pose_matrix)

    return np.array(poses)  # Retorna array (N, 4, 4)

#Função para salvar os dados aos poucos

In [ ]:
# Função para salvar os dados processados
def save_data_partially(train_data, test_data):
    with open(os.path.join(SAVE_PATH, "train_data.pkl").replace("\\", "/"), "wb") as f:
        pickle.dump(train_data, f)
    with open(os.path.join(SAVE_PATH, "test_data.pkl").replace("\\", "/"), "wb") as f:
        pickle.dump(test_data, f)
    print("✅ Dados salvos com sucesso!")

#Salvar os dados separados de Train e Test por cada sequencia

In [ ]:
# Carregar e dividir os dados em treino e teste, salvando progressivamente
def load_and_split_data():
    train_data = {}
    test_data = {}

    for seq in range(11):  # KITTI tem sequências de 00 a 10
        sequence = f"{seq:02d}"
        print(f"\n🔹 Processando sequência {sequence}...")

        images = load_images(sequence)
        lidar_data = load_lidar(sequence)
        poses = load_poses(sequence)

        # Separação em treino (00-07) e teste (08-10)
        if seq <= 7:
            train_data[sequence] = {"images": images, "lidar": lidar_data, "poses": poses}
        else:
            test_data[sequence] = {"images": images, "lidar": lidar_data, "poses": poses}

    save_data_partially(train_data, test_data)

#Salvar os dados separados direto de Train e Test

In [ ]:
# Função para carregar os dados processados
def load_saved_data():
    train_path = os.path.join(SAVE_PATH, "train_data.pkl").replace("\\", "/")
    test_path = os.path.join(SAVE_PATH, "test_data.pkl").replace("\\", "/")

    if os.path.exists(train_path) and os.path.exists(test_path):
        with open(train_path, "rb") as f:
            train_data = pickle.load(f)
        with open(test_path, "rb") as f:
            test_data = pickle.load(f)
        print("\n✅ Dados carregados com sucesso!")
        return train_data, test_data
    else:
        print("\n⚠️ Dados não encontrados. Processando novamente...")
        return None, None

#Rodando o pré-processamento

In [ ]:
# Carregar os dados se já existirem, senão processar e salvar
train_data, test_data = load_saved_data()

if train_data is None or test_data is None:
    load_and_split_data()